In [1]:
import numpy as np
from   sklearn.model_selection import train_test_split
import tensorflow as tf
from   tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D)
from   tensorflow.keras.losses import categorical_crossentropy as cxe

In [2]:
!ls ../../../../data/quickdraw

full_numpy_bitmap_ambulance.npy   full_numpy_bitmap_mermaid.npy
full_numpy_bitmap_crocodile.npy   full_numpy_bitmap_raccoon.npy
full_numpy_bitmap_eye.npy         full_numpy_bitmap_rifle.npy
full_numpy_bitmap_flamingo.npy    full_numpy_bitmap_snail.npy
full_numpy_bitmap_harp.npy        full_numpy_bitmap_stethoscope.npy


In [3]:
DATA = '../../../../data/quickdraw'
FILE_TEMPLATE = f'{DATA}/full_numpy_bitmap_%s.npy'
categories = ['ambulance', 'crocodile', 'eye', 'flamingo', 'harp', 
              'mermaid', 'raccoon', 'rifle', 'snail', 'stethoscope']
filenames = [FILE_TEMPLATE % x for x in categories]

In [4]:
BATCH = 128
DIM = 28
N_IMAGES = 100000 # reduce if mem issues
N_FILES = len(categories)
IMAGES_PER_CATEGORY = N_IMAGES // N_FILES
IMAGES_PER_CATEGORY

10000

### Preprocessing

In [5]:
i = 0
for path in filenames:
    x = np.load(path)
    x = x.astype('float32') / 255.
    y = [i] * len(x)
    x = x[:IMAGES_PER_CATEGORY]
    y = y[:IMAGES_PER_CATEGORY]
    if i == 0:
        x_all = x
        y_all = y
    else:
        x_all = np.concatenate((x, x_all), axis=0)
        y_all = np.concatenate((y, y_all), axis=0)
    i += 1

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=1103)

In [7]:
X_train = X_train.reshape(X_train.shape[0], DIM, DIM, 1)
X_test = X_test.reshape(X_test.shape[0], DIM, DIM, 1)
input_shape = (DIM, DIM, 1)

In [8]:
y_train = tf.keras.utils.to_categorical(y_train, N_FILES)
y_test = tf.keras.utils.to_categorical(y_test, N_FILES)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1103)

### ConvNet model

In [10]:
DROP = 0.25

In [11]:
mod = tf.keras.Sequential()

mod.add(Conv2D(
    32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
mod.add(MaxPooling2D(pool_size=(2, 2)))
mod.add(Dropout(DROP))

mod.add(Conv2D(64, (3, 3), activation='relu'))
mod.add(MaxPooling2D(pool_size=(2, 2)))
mod.add(Dropout(DROP))

mod.add(Flatten())
mod.add(Dense(128, activation='relu'))
mod.add(Dropout(2 * DROP))
mod.add(Dense(N_FILES, activation='softmax'))

In [12]:
mod.compile(loss=cxe, 
            optimizer=tf.keras.optimizers.Adadelta(), 
            metrics=['accuracy'])

In [13]:
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

In [14]:
EPOCHS = 25

In [17]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./tb_log_dir')]

In [19]:
mod.fit(X_train, 
        y_train,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1,
        validation_data=(X_valid, y_valid))

Train on 72000 samples, validate on 8000 samples
Epoch 1/25
72000/72000 [==============================] - 46s 643us/sample - loss: 2.2983 - accuracy: 0.1184 - val_loss: 2.2594 - val_accuracy: 0.1835
Epoch 2/25
72000/72000 [==============================] - 48s 661us/sample - loss: 2.2565 - accuracy: 0.1448 - val_loss: 2.2240 - val_accuracy: 0.2421
Epoch 3/25
72000/72000 [==============================] - 47s 654us/sample - loss: 2.2179 - accuracy: 0.1809 - val_loss: 2.1849 - val_accuracy: 0.3040
Epoch 4/25
72000/72000 [==============================] - 48s 668us/sample - loss: 2.1767 - accuracy: 0.2086 - val_loss: 2.1397 - val_accuracy: 0.3400
Epoch 5/25
72000/72000 [==============================] - 48s 664us/sample - loss: 2.1350 - accuracy: 0.2345 - val_loss: 2.0904 - val_accuracy: 0.3776
Epoch 6/25
72000/72000 [==============================] - 48s 661us/sample - loss: 2.0911 - accuracy: 0.2574 - val_loss: 2.0374 - val_accuracy: 0.4105
Epoch 7/25
72000/72000 [=====================

In [23]:
score = mod.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}\nTest Acc: {score[1]}')

Test loss: 1.4618729888916016
Test Acc: 0.5468999743461609


In [29]:
print(categories)
print('Predicted       Actual')
print('----------------------')

for _ in range(20):
    t = np.random.randint(len(X_test))
    x1 = X_test[t].reshape(1, DIM, DIM, 1)
    p = mod.predict(x1)
    print(f'{categories[np.argmax(p)]:15s} '
          f'{categories[np.argmax(y_test[t])]}')

['ambulance', 'crocodile', 'eye', 'flamingo', 'harp', 'mermaid', 'raccoon', 'rifle', 'snail', 'stethoscope']
Predicted       Actual
----------------------
snail           snail
eye             snail
crocodile       crocodile
raccoon         harp
ambulance       ambulance
rifle           rifle
eye             eye
harp            eye
ambulance       ambulance
snail           eye
raccoon         stethoscope
flamingo        stethoscope
rifle           rifle
flamingo        flamingo
eye             eye
snail           mermaid
rifle           rifle
harp            harp
harp            stethoscope
ambulance       mermaid


In [ ]:
#!tensorboard --logdir=./tb_log_dir